# Things to do
1. Write code  
2. Read bibliography (Parties, surface adhesion,...?)  
3. Vary parameters and make plots 
    -The first thing is to fix dt,L and D. Once they are selected they should not change!
4. Think on experiment 

# Description
Model for dynamical cluster formation in social events. This can also be seen as a preliminary work to study the formation of contact matrices. The dynamics for the individuals should satisfy:
- Be simple.
- Individual modeling with partial perception (collective behavior must emerge in a natural way).
- A Dynamical phase should emerge and be robust. No formation of giant component.
- The theory is neutral (there is no differences among the individuals).


# Things to discuss
Javi-About the model: I would stick with the easiest brownian motion as the mobility model for the agents. Despite it is not realistic (the movement of a person resembles more a persistent model such as the run and tumble model and there should be distant-dependent iterations), I think that is useful not to enter in these particular details. I don't think that our conclusions will depend on this particularities and moreover they could distract the reader. 

In [183]:
#Libraries
#%matplotlib notebook
import sys
from scipy.optimize import curve_fit
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time
import math 
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.patches as mpatches
%matplotlib inline


In [191]:
#Parameters
L=10 #Size of box LxL
N=20 #Number of agents
D=0.1 #Diffusion constant (FIX VALUE)
dt=0.01 #Time increment
sh=np.sqrt(D*dt)
tmax=5.0 #Run simulation until time tmax
itmax=int(round(tmax/dt))
r0=0.2 #If |r_i-r_j|<r0 then agents i and j forms a "social link" and stop moving
w=2.0 #1/w is the typical time scale for going to the bathroom  
pb=1.0-np.exp(-dt*w)
r=np.array([np.random.uniform(low=0,high=L,size=2) for i in range(N)]) #r[i]=(x_i,y_i)=position vector of agent i
v=np.zeros(N,dtype=int) #If v[i]=0, then agent i is moving and do not have a social link, if v[i]=1 then agent i
                        # cannot move and is part of a social community

In [192]:
#Functions
def Plot_state(save):
    '''
    Plot current state of system at iteration it.
    I recommend the creation of a folder named figures for saving the plots.
    If save=True then save fig in file, else just plot it.
    '''
    plt.title(r"$t=$"+str(it*dt))
    plt.xlim(0,L)
    plt.ylim(0,L)
    plt.scatter(r.T[0],r.T[1],alpha=0.5,color="salmon")
    for i in range(N):
        if v[i]==1:
            plt.scatter(r[i][0],r[i][1],alpha=1.0,color="skyblue") #Not moving agents
    if(save):
        plt.savefig("figures/snapshot_t_"+str(it)+".png",dpi=300)
    else:
        plt.show()
    plt.close()

def RBC(r):
    '''
    Implement Reflecting Boundary Conditions
    '''
    r[r>L]=2*L-r[r>L]
    r[r<0]=-r[r<0]
    
def move(r):
    '''
    If agent does not belong to a social link, then move.
    (not important code note: If agents do not freeze, I can move all of them with r+=sh*np.array([np.random.normal(size=2) for i in range(N)]))
    '''
    for i in range(N):
        if v[i]==0:
            r[i]+=sh*np.random.normal(size=2) #Integrate position
            #TO INCLUDE FORCE: r[i]+=dt*F(r)+sh*np.random.normal(size=2)
            
def createlink(v):
    '''
    Identify agents that are close enough to create a social link.
    Javi:I bet that this can be done in a more efficient way, but I don't think is very important.
    '''
    for i in range(N):
        for j in range(N):
            if ((i!=j) and (np.linalg.norm(r[i]-r[j]) <=r0)):
                v[i]=1
                v[j]=1
                
def destroylink(v):
    '''
    If an agent is in a social link, it can be destroyed with a constant rate w--> poissonian process
    '''
    for i in range(N):
        if v[i]==1:
            if np.random.uniform()<=pb: #Bathroom urgency
                v[i]=0
                r[i]=np.random.uniform(low=0,high=L,size=2) #Teleportation
                #Are there isolated nodes?
                for j in range(N):
                    if v[j]==1:
                        for k in range(N):
                            Isolated=True
                            if ((j!=k) and (np.linalg.norm(r[j]-r[k]) <=r0)):
                                Isolated=False
                                break
                            if (Isolated):
                                v[j]=0

                

In [193]:
#Main program
start = time.time()

for it in range(itmax):
    move(r) #r(t)->r(t+dt)
    RBC(r) # Reflecting boundary conditions
    destroylink(v) #Destroy social links
    createlink(v) #Create social links
    Plot_state(save=True) #This is the most expensive part of the code (3.35 mins for 999 frames)

end = time.time()
print("Time spend in runing main (mins)=",(end - start)/60)

Time spend in runing main (mins)= 1.8729427258173625
